In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext

conf = SparkConf()
conf.set("spark.master","local[4]")
conf.set("spark.driver.memory","16g")
conf.set("spark.app.name","sparkapp")
sc = SparkContext(conf=conf)
sqlc  =SQLContext(sc)

In [2]:
sqlc

In [55]:
df1 = sqlc.read.format("json").option("multiLine","True").load("/home/ubuntu/Notebooks/test.json")
df1.show()
df1.printSchema()

+---+----+-----+
| id|name|score|
+---+----+-----+
| 20| abc| null|
| 30| xyz|  4.5|
+---+----+-----+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- score: double (nullable = true)



In [53]:
rd1 = sc.textFile("/home/ubuntu/Datasets/ad-events_2014-01-20_00_domU-12-31-39-01-A1-34")
# print(rd1.take(1))
# print(rd1.count())
rd2 = rd1.map(lambda x:x.split(" 2014, "))
# print(rd2.take(1))
# print(rd2.map(lambda x:len(x)).take(50))
# print(rd2.filter(lambda x:len(x)==2).count())
rd3 = rd2.map(lambda x:x[1])
# print(rd3.take(1))
rd4 = rd3.map(lambda x:x.replace("}, {",","))
#{   },{   }--->
# print(rd4.take(4))

r1 = rd1.map(lambda x:x[:30]).map(lambda x:((str)(x),0))
r1.take(10)

from pyspark.sql.functions import *
df1 = sqlc.read.json(rd4)
# df1.show(5)
# len(df1.columns)
r1.take(10)
df2 = r1.toDF(["timezone","misc"]).drop("misc")
# df2.show(5)
df3 = df1.withColumn("extra1",monotonically_increasing_id())
df4 = df2.withColumn("extra2",monotonically_increasing_id())
resdf = df3.join(df4,col("extra1")==col("extra2"),"inner").drop("extra1","extra2")
# resdf.show(5)

In [61]:
from pyspark.sql import Row
r1 = Row(id=1,city="hyd")
r2 = Row(id=2,city="chennai")
d1 = sqlc.createDataFrame([r1,r2])
d1.show()

sqlc.read.json(d1.toJSON()).show()

+-------+---+
|   city| id|
+-------+---+
|    hyd|  1|
|chennai|  2|
+-------+---+

+-------+---+
|   city| id|
+-------+---+
|    hyd|  1|
|chennai|  2|
+-------+---+



In [64]:
import pandas

pdf = pandas.DataFrame({"name":["karna","hari","shankar"],"marks":[20,30,40]})
                      
print(pdf)

   marks     name
0     20    karna
1     30     hari
2     40  shankar


In [66]:
sdf = sqlc.createDataFrame(pdf)
sdf.show()


+-----+-------+
|marks|   name|
+-----+-------+
|   20|  karna|
|   30|   hari|
|   40|shankar|
+-----+-------+



In [68]:
pdf2 = sdf.toPandas()
pdf2
print(type(pdf2))


<class 'pandas.core.frame.DataFrame'>
